# RBF


We choose a gaußian activation function for a RBF neuron with it's derivatives $$y(x)=e^{-\sum\limits_i\frac{(x_i-c_i)^2}{\sigma_i^2}}$$
$$\frac d{dc}y(x)=2\sigma^2(x-c)y(x)$$
$$\frac d{d\sigma}y(x)=-2(x-c)\sigma y(x)$$
$$\frac d{dx}y(x)=-2\sigma^2(x-c)y(x)$$

Where $x,c,\sigma\in \mathbb{R}^n$

# Experiments


The MNIST data set can be downloaded in *.CSV* [this](https://github.com/pjreddie/mnist-csv-png/blob/master/process_mnist.py) git repository.

## RBF

### Boolean Function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import layer
from neurons import RBF
from typing import List
import model

TRAIN_RATE:float = 0.0001
TRAIN:bool = True
FIRST_LAYER_NEURONS:int = 3
SECOND_LAYER_NEURONS:int = 6
TRAINING_REPITIONS:int = 200
BATCH_SIZE:int = 4

layers:List[layer.DenseLayer] = []
layers.append(layer.DenseLayer(2, FIRST_LAYER_NEURONS, RBF()))
layers.append(layer.DenseLayer(FIRST_LAYER_NEURONS, SECOND_LAYER_NEURONS, RBF()))
layers.append(layer.DenseLayer(SECOND_LAYER_NEURONS, 1, RBF()))

m = model.Model(layers)

inputs = np.array([[0.0,0.0],[1.0,0.0],[0.0,1.0],[1.0,1.0]]).reshape(-1, BATCH_SIZE, 2)
expected = np.array([0.0, 1.0, 1.0, 0.0]).reshape(-1, BATCH_SIZE, 1)

if TRAIN:
    
    m.train(inputs, expected, TRAINING_REPITIONS)

    plt.plot(m.losses, label='Loss')
    plt.legend()

predicted_output = m.predict(inputs.reshape(1, 4, 2))

for i, output in enumerate(predicted_output.reshape(-1, 1)):
    print(f"Output: {output}\tExpected: {expected.flatten()[i]}")

pts = np.empty((FIRST_LAYER_NEURONS, 2))
colours = np.empty((FIRST_LAYER_NEURONS,3))
pts = layers[0].neurons.c
colours[:] = np.array([1.0,0.0,0.0])
plt.figure()
plt.scatter(pts[:,0], pts[:,1],c=colours)




In [ ]:
m.predict(inputs.reshape(1, 4, 2))

#### Results

* Even if the deviation can be trained, choosing a deviation that is too large, will result in odd behaviour.

  To give an example, if we're trying to learn the _AND_ function, the output will contain more zeros than ones. Using gradient decent it is better to decrease the error by creating many zeros which is done by moving away the center instead of lowering the deviation. In testing, this often results in getting in a state which is badly trained, but we cannot get out of.

* Training RBFs with multiple dense layers is less stable and will not converge as reliably using gradient decent. I believe this happens due to each RBF of lower levels 'prefering' a     different representation of the underlying feature space, and therefore no stable state is reached.

### MNIST

#### First test

In [ ]:
'''Load data'''
import numpy as np
from pathlib import Path
from typing import Tuple

BATCH_SIZE: int = 200

def load_mnist(path:Path) -> Tuple[np.ndarray, np.ndarray]:
    '''
    Returns:
        labels, images
    '''
    raw_data:np.ndarray = np.loadtxt(path, delimiter=",")
    indices = np.arange(raw_data.shape[0])
    np.random.shuffle(indices)
    return raw_data[indices,0].astype(int), raw_data[indices,1:] / 255.0

def filter_mnist(data, labels, filter_id):
    indices = np.where(labels==filter_id)
    return data[indices], labels[indices]

def create_one_hot_encoding(labels:np.ndarray)->np.ndarray:
    '''
    Creates a one hot encoding of the labels.

    Assumes the labels start from 0.
    '''
    indices = np.empty((2, labels.shape[0]), dtype=int)
    indices[0,:] = np.arange(labels.shape[0])
    indices[1,:] = labels
    num_labels = np.max(labels + 1)
    encoded_labels = np.zeros((labels.shape[0], num_labels))
    encoded_labels[indices[0], indices[1]] = 1.0
    #encoded_labels = np.where(labels == 0, 1.0, 0.0).reshape(-1,1)
    return encoded_labels

def create_batches(data:np.ndarray, labels:np.ndarray, batch_size:int) -> Tuple[np.ndarray, np.ndarray]:
    return data.reshape(-1, batch_size, data.shape[1]), labels.reshape(-1, batch_size, labels.shape[1])

train_labels, train_data = load_mnist(Path("mnist_train.csv"))
#train_data, train_labels = filter_mnist(train_data, train_labels, 0)
train_labels = create_one_hot_encoding(train_labels)

validation_labels, validation_data = load_mnist((Path("mnist_test.csv")))
validation_labels = create_one_hot_encoding(validation_labels)

train_data, train_labels = create_batches(train_data, train_labels, BATCH_SIZE)
validation_data, validation_labels = create_batches(validation_data, validation_labels, BATCH_SIZE)

In [ ]:
'''Create a model'''
from typing import List
import numpy as np
import layer
import model
from neurons import RBF
import loss

FIRST_LAYER_NEURONS:int = 512
SECOND_LAYER_NEURONS:int = 512
NUM_LABELS:int = train_labels.shape[2]

layers:List[layer.DenseLayer] = []
layers.append(layer.DenseLayer(train_data.shape[2], FIRST_LAYER_NEURONS, RBF()))
layers.append(layer.DenseLayer(FIRST_LAYER_NEURONS, SECOND_LAYER_NEURONS, RBF()))
layers.append(layer.DenseLayer(SECOND_LAYER_NEURONS, NUM_LABELS, RBF()))

m = model.Model(layers)

In [ ]:
'''Train the model.'''
from matplotlib import pyplot as plt

TRAINING_REPITIONS:int = 1
TRAIN_RATE:float = 0.0001

m.train(train_input=train_data, train_output=train_labels, epochs=TRAINING_REPITIONS, train_rate=TRAIN_RATE, loss_function=loss.quadratic_error)

In [ ]:
from matplotlib import pyplot as plt

plt.figure()
plt.plot(np.array(m.losses).flatten(), label="Loss")
plt.legend()
plt.show()

In [ ]:
print(f"Average validation error: {m.validate(validation_data, validation_labels)}.")

In [ ]:
def count_correct(input:np.ndarray, expected_labels:np.ndarray):
    output = m.predict(input).reshape(-1, 10)
    labels = np.argmax(output,axis=1)


    mask = labels == np.argmax(expected_labels.reshape(-1,10),axis=1)

    return np.sum(mask)
    #return np.sum(np.abs(np.where(m.predict(input).flatten() >= 0.5, 1.0, 0.0) - expected_labels.flatten()))

#print(f"Classified {count_correct(train_data, train_labels)} out of {train_data.shape[0] * train_data.shape[1]} training examples correctly.")
print(f"Classified {count_correct(validation_data, validation_labels)} out of {validation_data.shape[0] * validation_data.shape[1]} validation examples correctly.")

In [ ]:
from matplotlib import pyplot as plt

def visualize_prototypes(rbf_weights:np.ndarray):
    grey_scale_image = np.empty((*rbf_weights.shape, 3))
    grey_scale_image[:,0] = rbf_weights
    grey_scale_image[:,1] = rbf_weights
    grey_scale_image[:,2] = rbf_weights
    plt.figure()
    plt.imshow(grey_scale_image.reshape(28, 28, 3))
    plt.show()

visualize_prototypes(m.layers[0].neurons.c[1])